Setup Instructions
==============
These instructions need to be run once to teach Jupyter about the software environment we'd like to use for the tutorial. It's important that you've followed all the [setup steps](https://github.com/cms-jet/JMEDAS/blob/DAS2019/README.md#getting-started-setup), particularly the grid certificate instructions!

The following cell just needs to be executed once. If you logout and log back in, the setup will remain.

In [ ]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_10_6_4"
KERNEL_NAME="btv-exercise"

set -e
# Get the CMSSW libraries (specifically ROOT)
SCRAM_ARCH=$(ls -d /cvmfs/cms.cern.ch/$(/cvmfs/cms.cern.ch/common/cmsos)*/cms/cmssw/${CMSSW_VER} | tail -n 1 | awk -F / '{ print $4 }')
export SCRAM_ARCH
source /cvmfs/cms.cern.ch/cmsset_default.sh
if [ -d $CMSSW_VER ]; then
    rm -rf $CMSSW_VER
fi
cd $HOME/BTV_SW
scramv1 project CMSSW $CMSSW_VER 

cd $CMSSW_VER/src
eval `scramv1 runtime -sh`

git clone --recursive https://github.com/nhduongvn/CMSPOS-BTaggingExercise.git CMSPOS/BTaggingExercise

scram b -j 4
cd ..

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
cd $DIR
eval `scramv1 runtime -sh`
cd -
exec python "$@"
EOF
chmod +x bin/python_wrapper.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"

Results
=======

If successful, you should see something similar to the following:
`````
Submodule path 'CFIT': checked out '41670d3c78f053550700b0b75cfaa0b576b145b2'
>> Local Products Rules ..... started
>> Local Products Rules ..... done
>> Building CMSSW version CMSSW_10_6_4 ----
>> Subsystem BigProducts built
>> Entering Package CMSPOS/BTaggingExercise
>> Leaving Package CMSPOS/BTaggingExercise
>> Package CMSPOS/BTaggingExercise built
>> Subsystem CMSPOS built
gmake[1]: Entering directory '/home/cms.hnguyen/BTV_SW/CMSSW_10_6_4'
>> Local Products Rules ..... started
>> Local Products Rules ..... done
>> Creating project symlinks
>> Plugins of all types refreshed.
>> Done python_symlink
>> Compiling python modules cfipython/slc7_amd64_gcc820
>> Compiling python modules python
>> Done generating edm plugin poisoned information
>> All python modules compiled
gmake[1]: Leaving directory '/home/cms.hnguyen/BTV_SW/CMSSW_10_6_4'
Loaded CMSSW_10_6_4 into btv-exercise!
`````
The new kernel you just made will then show up in the various Jupyter dropdowns, adding it alongside the stock defaults.